In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# digitsデータを使うためにimportする
from torchvision.datasets import CIFAR10
import torchvision

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cpu


# 画像の読み込み

In [3]:
cd "C:/Users/kttn6/Google ドライブ/AI_Quest/gigu_quest_1"

C:\Users\kttn6\Google ドライブ\AI_Quest\gigu_quest_1


In [69]:
from skimage import io
import cv2
import glob
from tqdm import tqdm
from PIL import Image

#学習データの画像読み込み
file_list = glob.glob("./train/*.png")
image_list = []

for file in tqdm(file_list[0:50000]):
    #image = cv2.resize(io.imread(file), (224,224))
    image = cv2.imread(file)
    image = np.transpose(image, (2,0,1))
    #image = np.array(image)
    image_list.append(image)


100%|██████████| 50000/50000 [00:18<00:00, 2737.14it/s]


In [70]:
image_list = np.array(image_list)

In [80]:
image_list[:,:1].shape

(50000, 1, 32, 32)

In [ ]:
x = image_list.reshape(3, 4)

In [4]:
#iwaさんコード参考
PATH = 'C:/Users/kttn6/Google ドライブ/AI_Quest/'

import glob
import os.path as osp
import torch.utils.data as data
from torchvision import models, transforms
from PIL import Image


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                # transforms.RandomResizedCrop(
                #     resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                # transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)


def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = PATH + "gigu_quest_1/"
    target_path = osp.join(rootpath+phase+'/*.png')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list


class HymenopteraDataset(data.Dataset):
    """
    画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    """

    def __init__(self, file_list, y, transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.y = y #画像の正解ラベル
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        return img_transformed, y[index]


In [6]:
#学習データのラベルデータ読み込み
df = pd.read_csv('./train_master.tsv',sep='\t')
y = np.array(df["label_id"])
y

array([11, 15,  4, ...,  8,  7,  1], dtype=int64)

In [7]:
from torch.utils.data.dataset import Subset

# 1.3節で作成したクラスを同じフォルダにあるmake_dataset_dataloader.pyに記載して使用
#from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

#import sys
#sys.path.append('/content/drive/My Drive/Colab/my_project/my_modules')

# 画像へのファイルパスのリストを作成する
train_list = make_datapath_list(phase="train")
# test_list = make_datapath_list(phase="test")

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
    file_list=train_list, y=y, transform=ImageTransform(size, mean, std), phase='train')

C:/Users/kttn6/Google ドライブ/AI_Quest/gigu_quest_1/train/*.png


In [13]:
n_samples = len(train_dataset)
train_size = int(len(train_dataset) * 0.8)
val_size = n_samples - train_size

trainset, testset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [9]:
print(len(trainset),len(testset))

40000 10000


In [10]:
train_loader = DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = DataLoader(testset, batch_size=16, shuffle=True)

#### 転移学習へ進む

In [12]:
#Tensor型に変換
X = torch.tensor(image_list, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64) 

NameError: name 'image_list' is not defined

In [55]:
#Datasetの作成
trainval_dataset = torch.utils.data.TensorDataset(X, y)

TypeError: __init__() got an unexpected keyword argument 'transform'

In [10]:
#train_testに分割
n_samples = len(trainval_dataset)
train_size = int(len(trainval_dataset) * 0.8)
val_size = n_samples - train_size

trainset, testset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

In [11]:
int(train_size)

8000

In [12]:
from torchvision import datasets
from torch.utils.data.dataset import Subset

n_samples = len(trainval_dataset) # n_samples is 60000
train_size = int(n_samples * 0.8) # train_size is 48000

subset1_indices = list(range(0,train_size)) # [0,1,.....47999]
subset2_indices = list(range(train_size,n_samples)) # [48000,48001,.....59999]

trainset = Subset(trainval_dataset, subset1_indices)
testset   = Subset(trainval_dataset, subset2_indices)

print(len(trainset)) # 48000
print(len(testset)) # 12000

8000
2000


In [18]:
classes = tuple([i for i in range(20)])
len(classes)

20

In [14]:
train_loader = DataLoader(trainset, batch_size=16, shuffle=True)
test_loader = DataLoader(testset, batch_size=16, shuffle=True)

## 転移学習
---
今回はVGG16を利用しました。

In [15]:
# Enter your code here
model_ft2 = models.vgg16(pretrained=True)

In [16]:
# パラメーターを学習させないようにしている
# Enter your code here
for param in model_ft2.parameters():
    param.require_grad = False

In [19]:
# 最後の一層だけ書き換えて、最後の一層だけ学習させる
# Enter your code here
model_ft2.fc = nn.Linear(in_features=512, out_features=len(classes), bias=True)

In [20]:
loss_func = nn.CrossEntropyLoss()
print(loss_func)

CrossEntropyLoss()


In [21]:
optimizer = optim.SGD(model_ft2.parameters(), lr=0.001)

In [22]:
model_ft2 = model_ft2.to(device)

In [23]:
#練習用
lossfunc = loss_func
for epoch in range(1, 5+1):
        running_loss = 0
        running_train_loss = 0
        running_test_loss = 0

        # 学習モード
        net = model_ft2
        net.train()
        for step, batch  in enumerate(train_loader, 1):
            print('step{}'.format(step))
            net.train()
            xx, yy = batch[0].to(device), batch[1].to(device)
            xx = xx/255.0 - 0.5
           # 最後に計算した各パラメーターの勾配を初期化する
            optimizer.zero_grad()
            # フォワード計算を行う
            y_pred = net(xx)
            # 誤差関数を使ってギャップの計測
            loss = lossfunc(y_pred, yy)
            #print('X={}'.format(xx))
            print('Pred={}'.format(y_pred))
            
            #optimizer.zero_grad()
            # 誤差逆伝播法を使って自動微分
            loss.backward()
            # パラメーターを更新
            optimizer.step()
            
            

step1
Pred=tensor([[-1.1862,  0.2604, -1.9333,  ..., -1.7299,  0.8436,  0.9212],
        [-0.3256,  1.6480, -0.9734,  ..., -1.4681,  0.5530,  0.1676],
        [-1.3752,  1.2628, -1.3629,  ..., -1.7871, -0.1099,  0.5304],
        ...,
        [-0.4499,  1.8054, -2.0232,  ..., -1.0767,  1.1758,  0.1061],
        [-1.4916,  0.7220, -2.5096,  ..., -1.9371,  1.1820,  0.6104],
        [ 0.0978,  1.4153, -1.4650,  ..., -1.0984,  1.2834, -0.7878]],
       grad_fn=<AddmmBackward>)
step2
Pred=tensor([[-1.7887,  1.5471, -1.4597,  ..., -1.2858,  0.7582,  1.0331],
        [-1.0407,  0.9261, -0.5611,  ..., -0.6696,  0.8495,  0.4568],
        [-1.6561,  0.4278, -0.3513,  ..., -0.7516, -0.0926,  0.5404],
        ...,
        [-1.9640,  1.2012, -1.0064,  ..., -1.2520,  0.3237,  2.0046],
        [-0.6117,  1.4289,  0.4959,  ..., -0.8092,  0.2435,  0.3311],
        [-0.9588,  1.5859, -0.6434,  ..., -1.4679,  0.2523,  0.6877]],
       grad_fn=<AddmmBackward>)
step3
Pred=tensor([[-1.6868,  2.0165, -0.7049,

step18
Pred=tensor([[ 2.9678e-03,  3.8499e+00,  7.4854e-01,  ..., -5.6855e-01,
          9.4923e-01,  6.3046e-01],
        [ 4.9976e-01,  3.4551e+00,  1.6852e+00,  ...,  3.4551e-01,
          4.0110e-01,  1.9010e-01],
        [ 5.2233e-01,  3.5349e+00,  1.4863e+00,  ...,  9.2095e-01,
          1.3569e-01,  1.6468e-02],
        ...,
        [ 7.4775e-01,  4.4261e+00,  1.8179e+00,  ...,  2.1364e-01,
          1.0969e-01,  1.3880e+00],
        [ 1.6931e+00,  3.5366e+00,  1.7849e+00,  ...,  3.2885e-01,
          2.4870e-01,  2.9726e-01],
        [ 6.5340e-01,  4.6991e+00,  1.4826e+00,  ..., -4.9721e-01,
          2.7699e-01,  2.9631e-01]], grad_fn=<AddmmBackward>)
step19
Pred=tensor([[ 1.6365,  3.6587,  1.9001,  ...,  0.9034,  0.0930,  0.2771],
        [ 1.5717,  3.9112,  1.6933,  ...,  0.8182, -0.4192,  0.1047],
        [ 0.9736,  3.5619,  2.2516,  ..., -0.2115, -0.5091, -0.1897],
        ...,
        [ 1.9961,  3.6044,  0.9658,  ...,  0.2728,  0.9044,  0.7088],
        [ 2.1563,  4.8936,

step35
Pred=tensor([[ 3.2040e+00,  6.0800e+00,  5.1921e+00,  ..., -1.7961e-01,
         -7.8519e-02,  9.0186e-01],
        [ 1.7561e+00,  4.6544e+00,  2.8577e+00,  ..., -4.1620e-03,
          1.6684e-01,  1.4036e+00],
        [ 1.6230e+00,  5.1197e+00,  3.5906e+00,  ..., -3.2592e-01,
          2.7449e-01,  1.2684e+00],
        ...,
        [ 2.5631e+00,  4.6432e+00,  3.3624e+00,  ..., -2.1712e-01,
          3.4633e-01,  9.1727e-01],
        [ 2.7670e+00,  5.9563e+00,  3.7643e+00,  ..., -1.2215e-01,
         -3.7993e-01,  1.5384e+00],
        [ 2.2700e+00,  4.1537e+00,  3.1154e+00,  ...,  9.1214e-01,
         -8.9476e-01,  3.2113e-01]], grad_fn=<AddmmBackward>)
step36
Pred=tensor([[ 1.7736,  5.2420,  3.2757,  ...,  0.2218, -0.5041,  0.6926],
        [ 2.2670,  4.3984,  3.6950,  ...,  0.3856, -0.9066,  0.4797],
        [ 1.7323,  5.0455,  1.9332,  ...,  0.3433, -0.0365,  0.5829],
        ...,
        [ 1.0703,  4.3984,  2.4970,  ..., -0.3373, -0.2389,  0.2428],
        [ 1.7200,  4.0869,

KeyboardInterrupt: 

In [32]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch

print(torch.cuda.device_count())
# False

0


In [24]:
param = {
    'trainloader': train_loader,
    'valloader': test_loader,
    'net': model_ft2,
    'optimizer': optimizer,
    'lossfunc': loss_func,
    'epochs': 5,
    'device': device
    }

In [25]:
def training(trainloader, valloader, net, optimizer, lossfunc, epochs, device):
    train_loss_track = []
    test_loss_track = []

    for epoch in range(1, epochs+1):
        running_loss = 0
        running_train_loss = 0
        running_test_loss = 0

        # 学習モード
        net.train()
        for step, batch  in enumerate(trainloader, 1):
            # device = "cuda"の場合、GPUにデータを転送する
            xx, yy = batch[0].to(device), batch[1].to(device), #batchのtensor の中でX,yという配列で格納されている
            #print('xx{}'.format(xx))
            #print('yy{}'.format(yy))
            #xx, yy = batch[0], batch[1]
            
            #prev_model = model_ft2
            #rev_optimizer = optimizer
            
            # 最後に計算した各パラメーターの勾配を初期化する
            optimizer.zero_grad()
            # フォワード計算を行う
            y_pred = net(xx)
            # 誤差関数を使ってギャップの計測
            loss = lossfunc(y_pred, yy)
            print('prediction{}'.format(y_pred))
            
            #optimizer.zero_grad()
            # 誤差逆伝播法を使って自動微分
            loss.backward()
            # パラメーターを更新
            optimizer.step()
            # 学習データを使って損失を計算
            running_loss += loss.item()
            running_train_loss += loss.item()

            # 100 iterationごとにlossを出力
            if step % 100 == 0:
                print('[{:d} epoch, {:5d} iter] train/loss: {}'.format(epoch, step, running_loss / 100))
                running_loss = 0.0
        # エポックが終了したら平均損失を計算
        train_loss_track.append(running_train_loss / len(train_loader))

        # 評価（evaluation）モード
        net.eval()
        # 勾配計算用のパラメータを保存しない
        with torch.no_grad():
            for step, batch in enumerate(valloader, 1):
                # device = "cuda"の場合、GPUにデータを転送する
                xx_test, yy_test = batch[0].to(device), batch[1].to(device)
                #xx_test, yy_test = batch[0], batch[1]
                # 予測値を計算
                y_pred = net(xx_test)
                # 誤差関数を使ってギャップの計測
                test_loss = lossfunc(y_pred, yy_test)
                # テストデータを使って損失を計算
                running_test_loss += test_loss.item()
        # 誤差をトラッキング
        test_loss_track.append(running_test_loss / len(valloader))
        print('val/loss: {}'.format(running_test_loss / len(valloader)))
    return net, train_loss_track, test_loss_track

In [26]:
model_ft2, train_loss_track, test_loss_track = training(**param)

predictiontensor([[ 0.1718,  1.5973,  3.6517,  ..., -0.5247, -1.1722,  1.4372],
        [-0.7109,  1.6782, -0.2509,  ..., -3.3883, -0.9128,  1.3525],
        [ 1.0328,  2.6343,  0.3135,  ..., -0.4509,  0.2642,  3.3603],
        ...,
        [ 1.3575,  2.8012,  3.4027,  ..., -0.9443, -0.5447,  1.7857],
        [ 0.6019,  1.5006,  2.0185,  ..., -3.0108, -0.4060,  2.5505],
        [-3.5004, -0.8450, -1.2210,  ..., -1.7256, -2.6423,  4.0673]],
       grad_fn=<AddmmBackward>)
predictiontensor([[ 0.8030,  1.6257,  3.5304,  ..., -2.2327,  1.6124, -0.6394],
        [ 0.5029,  1.7032,  1.7462,  ..., -2.6410,  0.6951, -2.1919],
        [ 1.9527,  2.0926,  2.0758,  ..., -2.8828,  0.3324, -1.5571],
        ...,
        [-0.1709,  2.1385,  2.2761,  ..., -2.9661, -0.5804, -1.6920],
        [ 0.3188,  0.9138,  0.5355,  ..., -2.8859,  1.0315, -2.2853],
        [ 0.1958,  1.8650,  1.3048,  ..., -2.2152,  0.5784, -2.4533]],
       grad_fn=<AddmmBackward>)
predictiontensor([[ 6.7271,  7.8893,  6.7990,  .

predictiontensor([[ 3.8567,  7.8820,  4.7903,  ...,  0.9702,  0.6170,  1.4037],
        [ 4.5524,  7.9428,  7.1001,  ...,  1.2726,  0.5620,  0.8046],
        [ 2.5715,  5.5840,  5.3133,  ...,  0.2613,  0.0101,  1.8276],
        ...,
        [ 3.4829,  5.7726,  4.3763,  ...,  1.6317,  0.3305,  1.6551],
        [ 2.4527,  7.4752,  4.5779,  ...,  1.1570, -0.6792,  1.1433],
        [ 1.8161,  4.8056,  3.5366,  ...,  1.1326, -0.2686,  0.9589]],
       grad_fn=<AddmmBackward>)
predictiontensor([[ 4.1827,  5.9679,  3.0886,  ...,  0.6385,  0.2729,  0.2595],
        [ 2.4416,  5.1052,  4.7078,  ..., -0.9333,  0.1902,  0.8431],
        [ 6.1815,  9.8610,  4.7265,  ...,  1.3164,  0.1508,  0.4958],
        ...,
        [ 3.0326,  6.4338,  4.4601,  ...,  0.4591, -0.1807, -0.5072],
        [ 2.8569,  6.0742,  6.1112,  ..., -0.0120, -1.1586,  1.4014],
        [ 4.4502,  6.1745,  4.8814,  ..., -0.0881, -0.0387,  1.1055]],
       grad_fn=<AddmmBackward>)
predictiontensor([[ 5.0821,  6.4779,  4.7034,  .

KeyboardInterrupt: 

In [43]:
model_ft2

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 交差エントロピー誤差の推移

In [ ]:
plt.plot(train_loss_track)

In [ ]:
plt.plot(test_loss_track)

In [ ]:
def calculate_accuracy(testloader, net):
    true = 0
    total = 0

    all_labels = np.array([])
    all_preds = np.array([])

    net.eval()
    with torch.no_grad():
        for test_xx, test_yy in testloader:
            
            # device = "cuda"の場合、GPUにデータを転送する
            test_xx = test_xx.to(device)
            test_yy = test_yy.to(device)

            outputs = net(test_xx)
            _, predicted = torch.max(outputs.data, 1)
            
            all_labels = np.append(all_labels, test_yy.cpu().data.numpy())
            all_preds = np.append(all_preds, predicted.cpu().numpy())
            
            total += test_yy.size(0)
            true += (predicted == test_yy).sum().item()
    print('Accuracy: {:.2f} %'.format(100 * float(true/total)))

    return all_labels, all_preds

In [ ]:
all_labels, all_preds = calculate_accuracy(test_loader, model_ft2)

## testデータで混同行列を作成

In [ ]:
def plot_conf_mat(all_labels, all_preds):
    labels = np.unique(all_labels)
    cm = confusion_matrix(all_labels, all_preds, labels=labels)
    cm_labeled = pd.DataFrame(cm, columns=labels, index=labels)
    return cm_labeled

plot_conf_mat(all_labels, all_preds)